# This is a slideshow

Use RISE to run this notebook as a slideshow
    
<http://rise.readthedocs.io>

![](rise-icon.png)

# Warning

If you recently installed or upgraded jupyter, you may have tornado version 5.x, and that currently (04/18) does not work.

`pip3 install tornado==4.5.3`

See also <https://r2lab.inria.fr/school.md#2-note-on-using-notebooks>

# Intro to `asyncio` and `asynciojobs`

# With *python >= 3.5* : `asyncio`

In [ ]:
# from the standard library
import asyncio

Within this frame work you can define **coroutines**

In [ ]:
async def duration(message, delay=0.5):
     print(">>", message)
     await asyncio.sleep(delay)
     print("<<", message)    

# All coroutines need to run through an event loop

In [ ]:
mainloop = asyncio.get_event_loop()

In [ ]:
mainloop.run_until_complete(
    duration("Hello world")
)

# Stuff runs ***in parallel  in a single thread*** !

In [ ]:
mainloop.run_until_complete(
    asyncio.gather(
        duration("one second", 1),
        duration("two seconds", 2)
    )
)

# `asynciojobs` is about *dependencies*

`asyncio` does not have a native notion to express things like 

> I need such and such tasks to have completed before I run this

and that's what `asynciojobs` is about

# `asynciojobs` 

Not part of the standard library

`pip3 install asynciojobs`

**Note** that this comes as a dependency if you directly install `apssh` which we will see next

`pip3 install apssh`

# The menagerie inside `asynciojobs`

Essentially 2 classes:

In [ ]:
from asynciojobs import Scheduler, Job

# Simplest example

In [ ]:
s = Scheduler()

In [ ]:
Job(
    duration("job1 - one second", 1),
    label = 'job1',
    scheduler = s, 
)

In [ ]:
#
Job(
    duration("job2 - two seconds", 2),
    label = 'job2',
    scheduler = s, 
)

In [ ]:
s.graph()

In [ ]:
s.run()

# Roles inside `asynciojobs`  

* a `Job` object typically is an atomic coroutine
* a `Scheduler` object contains jobs

* jobs have a `required` relationship between them
   * must be acyclic of course
* scheduler acts as a mainloop

In [ ]:
# Sequence

In [ ]:
s = Scheduler()

In [ ]:
job1 = Job(
    duration("job1 - one second", 1),
    label = 'job1',
    scheduler = s, 
)

In [ ]:
#
Job(
    duration("job2 - two seconds", 2),
    label = 'job2',
    required = job1,
    scheduler = s, 
)

In [ ]:
s.graph()

In [ ]:
s.run()

# etc..

In [ ]:
s = Scheduler()

In [ ]:
job1 = Job(
    duration("job1 - 0.5s", .5),
    label = 'job1',
    scheduler = s, 
)

In [ ]:
job2 = Job(
    duration("job2 - two seconds", 2),
    label = 'job2',
    required = job1,
    scheduler = s, 
)

In [ ]:
job3 = Job(
    duration("job3 - one second", 1),
    label = 'job3',
    required = job1,
    scheduler = s, 
)

In [ ]:
job4 = Job(
    duration("job4 - .2s", .2),
    label = 'job2',
    required = (job2, job3),
    scheduler = s, 
)

In [ ]:
s.graph()

In [ ]:
s.run()